In [1]:
import sqlite3
import json

In [2]:
db = sqlite3.connect('../backend/.tmp/data.db')

## Populate Ingredients

In [61]:
ingredients = json.load(open('data/items.json'))['items']

In [62]:
for ingredient in ingredients:
    ingredient['ingredientname'] = ingredient.pop('name')
    ingredient['type'] = ingredient['type'].replace(' ','_').replace('é','e').replace('è','e')

In [64]:
c = db.cursor()

for ingredient in ingredients:
    table = "ingredients(ingredientname,type)"
    query = """
        INSERT OR IGNORE INTO 
            {} 
        VALUES 
            (?, ?)""".format(table) 
    c.execute(query, (ingredient['ingredientname'], ingredient['type']))

c.execute("SELECT * FROM ingredients")
resultat  = c.fetchall()
c.close()

In [65]:
resultat

[(1, 'saumon', 'poisson', '2020-05-22 13:39:21', '2020-05-22 13:39:21'),
 (2, 'aneth', 'condiment', '2020-05-22 13:39:21', '2020-05-22 13:39:21'),
 (3,
  'huile d olive',
  'condiment',
  '2020-05-22 13:39:21',
  '2020-05-22 13:39:21'),
 (4, 'poulet', 'volaille', '2020-05-22 13:39:21', '2020-05-22 13:39:21'),
 (5,
  'crème fraiche',
  'produit_laitier',
  '2020-05-22 13:39:21',
  '2020-05-22 13:39:21'),
 (6, 'curry', 'condiment', '2020-05-22 13:39:21', '2020-05-22 13:39:21'),
 (7,
  'burritos',
  'saveurs_du_monde',
  '2020-05-22 13:39:21',
  '2020-05-22 13:39:21'),
 (8, 'viande hachée', 'viande', '2020-05-22 13:39:21', '2020-05-22 13:39:21'),
 (9, 'tomates', 'legume', '2020-05-22 13:39:21', '2020-05-22 13:39:21'),
 (10, 'poivrons', 'legume', '2020-05-22 13:39:21', '2020-05-22 13:39:21'),
 (11, 'courgettes', 'legume', '2020-05-22 13:39:21', '2020-05-22 13:39:21'),
 (12, 'salade', 'legume', '2020-05-22 13:39:21', '2020-05-22 13:39:21'),
 (13, 'cabillaud', 'poisson', '2020-05-22 13:39:21

In [66]:
#db.commit()

In [58]:
c = db.cursor()
#c.execute("DELETE FROM ingredients")
c.execute("SELECT * FROM ingredients")
resultat  = c.fetchall()
c.close()

In [59]:
resultat

[]

## Populate Recipes

In [5]:
recipes = json.load(open('data/bouffe.json'))['menu']

In [6]:
from slugify import slugify

c = db.cursor()

for recipe in recipes:
    slug = slugify(recipe['meal'])
    table = "recipes(recipename,slug)"
    query = """
        INSERT OR IGNORE INTO 
            {} 
        VALUES 
            (?, ?)""".format(table) 
    c.execute(query, (recipe['meal'], slug))

c.execute("SELECT * FROM recipes")
resultat  = c.fetchall()
c.close()

In [7]:
resultat

[(1,
  'Riz poulet au curry',
  'riz-poulet-curry',
  None,
  None,
  1590145740831,
  1590145740831),
 (2,
  'Poulet au curry (riz)',
  'poulet-au-curry-riz',
  None,
  None,
  '2020-05-22 13:58:20',
  '2020-05-22 13:58:20'),
 (3,
  'Poulet au curry (pâtes)',
  'poulet-au-curry-pates',
  None,
  None,
  '2020-05-22 13:58:20',
  '2020-05-22 13:58:20'),
 (4,
  'Poulet crème champignons (riz)',
  'poulet-creme-champignons-riz',
  None,
  None,
  '2020-05-22 13:58:20',
  '2020-05-22 13:58:20'),
 (5,
  'Poulet crème champignons (pâtes)',
  'poulet-creme-champignons-pates',
  None,
  None,
  '2020-05-22 13:58:20',
  '2020-05-22 13:58:20'),
 (6,
  'Poulet ratatouille',
  'poulet-ratatouille',
  None,
  None,
  '2020-05-22 13:58:20',
  '2020-05-22 13:58:20'),
 (7,
  'Dinde a la tomate aux petits oignons',
  'dinde-a-la-tomate-aux-petits-oignons',
  None,
  None,
  '2020-05-22 13:58:20',
  '2020-05-22 13:58:20'),
 (8,
  'Nuggets',
  'nuggets',
  None,
  None,
  '2020-05-22 13:58:20',
  '2020-0

In [8]:
#db.commit()

In [22]:
c = db.cursor()
name = 'Poulet au curry (riz)'
t = c.execute("SELECT rowid FROM recipes WHERE recipename = ?", (name,))
print(t.fetchall()[0][0])
c.close()

2


## Populate Recipe Component (Ingredients)

In [92]:
db.rollback()

In [93]:
c = db.cursor()

for recipe in recipes:
    r = c.execute("SELECT rowid FROM recipes WHERE recipename = ?", (recipe['meal'],))
    recipeid = r.fetchall()[0][0]
    orders = 0
    for ingredient in recipe['groceries']:
        orders += 1
        t = c.execute("SELECT rowid FROM ingredients WHERE ingredientname = ?", (ingredient,))
        ingredientid = t.fetchall()[0][0]
        table_components = "components_category_ingredients (ingredient)"
        query_components = """
            INSERT OR IGNORE INTO 
                {} 
            VALUES 
                (?)""".format(table_components)
        c.execute(query_components, (ingredientid,))
        component_id = c.lastrowid
        table_recipes_components = 'recipes_components (field,`order`,component_type,component_id,recipe_id)'
        query_recipes_components = """
            INSERT INTO 
                {} 
            VALUES 
                (?, ?, ?, ?, ?)""".format(table_recipes_components)
        field = "ingredients"
        component_type = "components_category_ingredients"
        c.execute(query_recipes_components, (field,orders,component_type,component_id,recipeid))

c.close()

In [94]:
c.lastrowid

166

In [95]:
c = db.cursor()
c.execute("SELECT * FROM components_category_ingredients")
resultat  = c.fetchall()
c.close()

In [96]:
resultat

[(1, None, None, 4),
 (2, None, None, 6),
 (3, None, None, 5),
 (4, None, None, 14),
 (5, None, None, 4),
 (6, None, None, 5),
 (7, None, None, 6),
 (8, None, None, 14),
 (9, None, None, 4),
 (10, None, None, 5),
 (11, None, None, 6),
 (12, None, None, 63),
 (13, None, None, 4),
 (14, None, None, 5),
 (15, None, None, 49),
 (16, None, None, 14),
 (17, None, None, 4),
 (18, None, None, 5),
 (19, None, None, 49),
 (20, None, None, 63),
 (21, None, None, 4),
 (22, None, None, 15),
 (23, None, None, 59),
 (24, None, None, 47),
 (25, None, None, 60),
 (26, None, None, 14),
 (27, None, None, 35),
 (28, None, None, 36),
 (29, None, None, 61),
 (30, None, None, 59),
 (31, None, None, 9),
 (32, None, None, 20),
 (33, None, None, 10),
 (34, None, None, 12),
 (35, None, None, 7),
 (36, None, None, 8),
 (37, None, None, 9),
 (38, None, None, 20),
 (39, None, None, 10),
 (40, None, None, 12),
 (41, None, None, 23),
 (42, None, None, 8),
 (43, None, None, 24),
 (44, None, None, 75),
 (45, None, None

In [97]:
c = db.cursor()
c.execute("SELECT * FROM recipes_components")
resultat  = c.fetchall()
c.close()

In [98]:
resultat

[(5, 'ingredients', 1, 'components_category_ingredients', 1, 1),
 (6, 'ingredients', 2, 'components_category_ingredients', 2, 1),
 (7, 'ingredients', 4, 'components_category_ingredients', 4, 1),
 (8, 'ingredients', 3, 'components_category_ingredients', 3, 1),
 (9, 'ingredients', 1, 'components_category_ingredients', 5, 2),
 (10, 'ingredients', 2, 'components_category_ingredients', 6, 2),
 (11, 'ingredients', 3, 'components_category_ingredients', 7, 2),
 (12, 'ingredients', 4, 'components_category_ingredients', 8, 2),
 (13, 'ingredients', 1, 'components_category_ingredients', 9, 3),
 (14, 'ingredients', 2, 'components_category_ingredients', 10, 3),
 (15, 'ingredients', 3, 'components_category_ingredients', 11, 3),
 (16, 'ingredients', 4, 'components_category_ingredients', 12, 3),
 (17, 'ingredients', 1, 'components_category_ingredients', 13, 4),
 (18, 'ingredients', 2, 'components_category_ingredients', 14, 4),
 (19, 'ingredients', 3, 'components_category_ingredients', 15, 4),
 (20, 'in

In [99]:
#db.commit()